In [88]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Subset, DataLoader
import torchvision
import torchvision.transforms as transforms
import numpy as np
import tqdm
from sklearn.metrics import classification_report
import pandas as pd

In [54]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

num_epochs = 15
batch_size = 128
learning_rate = 0.001

In [48]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))  # mean and std of MNIST
])

train_dataset = torchvision.datasets.MNIST(root="./data", train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root="./data", train=False, transform=transform)

In [27]:
np.unique(train_dataset.targets, return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([5923, 6742, 5958, 6131, 5842, 5421, 5918, 6265, 5851, 5949]))

As we see, initially, the dataset is more or less balanced. We will take 10% of each classes to the new balanced dataset. The unbalanced dataset will consist of 100% samples from type 0, and 10% of samples from other types

In [79]:
indices = [np.where(np.array(train_dataset.targets) == 0)[0][:5500:20]]
indices_balanced = [np.where(np.array(train_dataset.targets) == 0)[0][:5000:100]]
for i in range(1, 10):
    indices.append(np.where(np.array(train_dataset.targets) == i)[0][:5000:200])
    indices_balanced.append(np.where(np.array(train_dataset.targets) == i)[0][:5000:100])
indices = np.concatenate(indices)
indices_balanced = np.concatenate(indices_balanced)
train_dataset_balanced = Subset(train_dataset, indices_balanced)
train_dataset_imbalanced = Subset(train_dataset, indices)
len(train_dataset_balanced), len(train_dataset_imbalanced)

/tmp/ipykernel_19102/3866804858.py:1: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  indices = [np.where(np.array(train_dataset.targets) == 0)[0][:5500:20]]
/tmp/ipykernel_19102/3866804858.py:2: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  indices_balanced = [np.where(np.array(train_dataset.targets) == 0)[0][:5000:100]]
/tmp/ipykernel_19102/3866804858.py:4: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ mu

(500, 500)

In [87]:
class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(28*28, 128)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)

    def forward(self, x):
        x = x.view(-1, 28*28)  # flatten
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x


def get_net():
    return NeuralNet().to(device)


def train(net: NeuralNet, train_loader, epochs):
        # Loss and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(net.parameters(), lr=learning_rate)

    # Training loop
    for epoch in tqdm.tqdm(range(epochs)):
        net.train()
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            outputs = net(images)
            loss = criterion(outputs, labels)

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

    return net


def eval(net, test_loader):
    all_labels = []
    all_predictions = []
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs, 1)
            all_predictions.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    return classification_report(all_labels, all_predictions, output_dict=True)


def avg_report(reports):
    res = {}
    for entry in zip(*[r.items() for r in reports]):
        if isinstance(entry[0][1], float):
            res[entry[0][0]] = sum(t[1] for t in entry) / len(reports)
        else:
            res[entry[0][0]] = avg_report([t[1] for t in entry])
    return res

In [89]:
EXPERIMENT_REPEATS = 20

train_loader_balanced = DataLoader(train_dataset_balanced, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

stats = []
for _ in range(EXPERIMENT_REPEATS):
    net = get_net()
    train(net, train_loader_balanced, epochs=num_epochs)
    stats.append(eval(net, test_loader))

100%|██████████| 15/15 [00:01<00:00, 13.88it/s]


In [90]:
stats = avg_report(stats)

In [92]:
pd.DataFrame(stats)

,0,1,2,3,4,5,6,7,8,9,accuracy,macro avg,weighted avg
precision,0.936353,0.929458,0.860811,0.788146,0.864094,0.819354,0.910480,0.905956,0.773553,0.828178,0.8603,0.861638,0.862898
recall,0.921990,0.942159,0.789099,0.842129,0.886660,0.758240,0.893633,0.853648,0.852823,0.846234,0.8603,0.858661,0.860300
f1-score,0.928969,0.935721,0.822889,0.813544,0.874834,0.786744,0.901871,0.878772,0.810106,0.836368,0.8603,0.858982,0.860451
support,980.000000,1135.000000,1032.000000,1010.000000,982.000000,892.000000,958.000000,1028.000000,974.000000,1009.000000,0.8603,10000.000000,10000.000000


In [93]:
train_loader_unbalanced = DataLoader(train_dataset_imbalanced, batch_size=batch_size, shuffle=True)

stats = []
for _ in range(EXPERIMENT_REPEATS):
    net = get_net()
    train(net, train_loader_unbalanced, epochs=num_epochs)
    stats.append(eval(net, test_loader))
stats = avg_report(stats)
pd.DataFrame(stats)

100%|██████████| 15/15 [00:00<00:00, 16.14it/s]


,0,1,2,3,4,5,6,7,8,9,accuracy,macro avg,weighted avg
precision,0.767910,0.889459,0.866157,0.737081,0.792943,0.819030,0.876982,0.897930,0.759974,0.767212,0.8142,0.817468,0.818721
recall,0.991276,0.958414,0.781056,0.795594,0.770927,0.650448,0.875418,0.803405,0.738860,0.744995,0.8142,0.811039,0.814200
f1-score,0.865232,0.922520,0.820755,0.763951,0.779731,0.723541,0.876042,0.847299,0.747866,0.752968,0.8142,0.809990,0.812313
support,980.000000,1135.000000,1032.000000,1010.000000,982.000000,892.000000,958.000000,1028.000000,974.000000,1009.000000,0.8142,10000.000000,10000.000000


As we see, in specific scenarios class imbalance leads to performance downgrade when comparing with dataset of the same size but equal class distribution. However, the recall in case of overrepresented class is lower in balanced scenario.

In [96]:
indices_ext = [np.where(np.array(train_dataset.targets) == 0)[0][:4100:10]]
for i in range(1, 10):
    indices_ext.append(np.where(np.array(train_dataset.targets) == i)[0][:5000:500])
indices_ext = np.concatenate(indices_ext)
train_dataset_unbalanced_ext = Subset(train_dataset, indices_ext)
train_loader_unbalanced_ext = DataLoader(train_dataset_unbalanced_ext, batch_size=batch_size, shuffle=True)
len(train_dataset_unbalanced_ext)

/tmp/ipykernel_19102/1932239961.py:1: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  indices_ext = [np.where(np.array(train_dataset.targets) == 0)[0][:4100:10]]
/tmp/ipykernel_19102/1932239961.py:3: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  indices_ext.append(np.where(np.array(train_dataset.targets) == i)[0][:5000:500])


500

In [97]:
stats = []
for _ in range(EXPERIMENT_REPEATS):
    net = get_net()
    train(net, train_loader_unbalanced_ext, epochs=num_epochs)
    stats.append(eval(net, test_loader))
stats = avg_report(stats)
pd.DataFrame(stats)

100%|██████████| 15/15 [00:00<00:00, 15.14it/s]


,0,1,2,3,4,5,6,7,8,9,accuracy,macro avg,weighted avg
precision,0.434724,0.768834,0.828723,0.744925,0.611503,0.862751,0.877502,0.808639,0.699019,0.485100,0.650885,0.712172,0.711858
recall,0.997959,0.958943,0.557122,0.614356,0.603157,0.361267,0.580219,0.644747,0.627618,0.498018,0.650885,0.644341,0.650885
f1-score,0.604535,0.851609,0.659446,0.670712,0.600033,0.496335,0.695098,0.715710,0.653211,0.486029,0.650885,0.643272,0.647724
support,980.000000,1135.000000,1032.000000,1010.000000,982.000000,892.000000,958.000000,1028.000000,974.000000,1009.000000,0.650885,10000.000000,10000.000000
